http://blog.naver.com/PostView.nhn?blogId=sogangori&logNo=221087066947

속도는 빠르지만 성능은 낮았다. 우리는 그 이유가 극단적인 클래스 불균형 문제 때문이라는 것을 발견했다.
잘 찾은 class에 대해서는 loss를 적게 줘서 loss 갱신을 거의 하지 못하게 하고 잘 찾지 못한 class에 대해서는 loss를 크게 줘서 loss 갱신을 크게 하는 것입니다
그래서 쉽게 분류되는 대부분의 negative 샘플들에 의해서 학습이 압도 되는 문제를 해결 할 수 있다.
또한 Focal Loss를 사용하면 one-stage Network에서도 two-stage Network 이상의 성능을 얻을 수 있다.
RetinaNet 은 one-stage detector 만큼 빠르면서도 기존의 모든 최고 성능의 detector들을 능가하는 정확도를 보여준다.


### 1. Introduction

현재 최고의 성능을 내는 object detecots 는 two-stage 기반의 proposal-driven 방식이다.
R-CNN 방식은 후보 오브젝트 위치들의 sparse set 을 예측하고 후보 위치들의 오브젝트의 클래스를 분류한다. 
그동안 COCO benchmark에서는 이러한 two-stage 프레임워크가 줄곧 top 의 위치를 지켜왔다.


1-stage detector 는 영상 전체 위치에서 오브젝트의 위치, 크기, 비율을 dense 하게 샘플링한다.
이 방식으로 YOLO, SSD 등은 최고 성능의 two-stage 방식에 비해 굉장히 빠르지만 성능면에서는 단지 10~40% 부족하다.

COCO AP 에 대해서 기존 최고 성능의 모델은 Feature Pyramid Network(FPN)이나 Faster R-CNN의 변형인 Mask R-CNN 과 같은 two-stage detectors 였다.

우리는 one-stage detector의 학습과정에서 클래스 불균형이 가장 큰 문제였음을 확인하였다.
그래서 새로운 로스 함수를 제안하여 이 문제의 장벽을 제거함으로써 이러한 좋은 성능을 낼 수 있었다.



R-CNN 계열의 detectors 에서는 two-stage cascade나 sampling heuristics을 사용해서 클래스 불균형 문제를 해결했다.

proposal 단계에서(Selective Search, EdgeBoxes, DeepMask, RPN) 후보들의 대부분인 배경 샘플들을 필터링한다.

두번째 분류 단계에서는 foreground-to-background ratio가 1:3이 되도록하거나 online hard example mining(OHEM)등의 sampling heuristic 방법을 사용한다.

이러한 샘플링 작업들이 foreground외 background 클래스의 균형을 맞춰서 학습이 잘되게 해준다.


반면 one-stage detector는 영상 전체에서 후보 오브젝트가 매우 많이 나온다.
학습과정에서 loss가 너무나 많은 background examples에 의해 압도되어 버리기 때문에 R-CNN에서 사용하던 샘플링 기법을 적용하는 것만으로는 충분하지 않다.


이 논문에서는 새로운 로스 함수를 사용하여 이전의 방법들에 비해 훨씬 효율적으로 클래스 불균형 문제를 해결하였다.
우리의 로스 함수는 동적(Dynamic)으로 크로스 엔트로피 로스의 크기를 조절한다. 

<img src='./imgs/retina.png'>

그림 1에서 나타나듯이 클래스 분류가 잘 된 예제에 대해서는 scaling factor 가 0 에 가까워진다.
직관적으로 scaling factor는 자동으로 학습과정에서 분류하기 쉬운 예제(easy example)들이 학습에 기여하는 정도를 낮춘다(down-weight).

반면 분류하기 어려운 예제(hard example)들에 대해서는 매우 집중한다.


실험의 결과를 통해서 우리가 제안하는 Focal Loss 가 one-stage detector 에서 매우 높은 성능을 낼 수 있으며, 기존의 sampling heuristic 이나 hard example mining 등에 대한 효율적인 대안이 될 수 있다는 것을 보여준다.

focal loss의 효과를 보여주기 위해서, 우리는 간단한 one-stage object detector 인 RetinaNet을 디자인했다.
입력 이미지로부터 object locations을 dense sampling 한다는 의미에서 RetinaNet이라고 이름을 지었다.
디자인면에서 효율적으로 in-network feature pyramid 및 앵커 박스를 사용하는 것이 특징이다.

RetinaNet 은 효율적이고 정확하다.
ResNet-101-FPN backbone 으로 하는 우리의 최고 성능의 모델에서 COCO test-dev AP 39.1 , 5 fps 의 성능을 낸다.

<img src='./imgs/retina2.png'>

그림 2에서 나타난것처럼, 지금까지 published 된 모든 single-model 과 모든 two-stage ectectors 의 성능을 능가한다. 그래프에는 다양한 입력 크기를 받는(400-800 픽셀) ResNet-50-FPN(파란색 원)과 ResNet-101-FPN(오렌지 다이아몬드) 을 바탕으로 만든 레티나넷의 속도와 성능간 관계가 나타나있다. 25 미만을 뺀 AP 성능에서 RetinaNet은 지금까지의 어떠한 모델들보다도 성능보다 좋다.
그래프에는 나타나있지 않지만 가장 높은 성능을 보인 RetinaNet 의 AP 는 39.1이다.



$\gamma>0$ 으로 셋팅하면 상대적으로 잘 분류된 예제들(well-classified examples: Pt> 0.5)의 로스를 줄일 수 있다.

그러면 잘못 분류된 예제들에게 더욱 집중할 수 있게 된다. 

대부분이 구분하기 쉬운 배경 예제들이 많은 경우 포컬 로스를 사용하면 높은 성능을 갖는 dense object detector 를 학습시킬 수 있다.




### 2. Related Work

* Classic Object Detectors: 

슬라이딩 윈도우, HOG, DPMs 등은 그동안 효율적으로 사용되어 왔다.
그동안 슬라이딩-윈도우 방식이 detection 패러다임을 이끌었지만 딥러닝의 부활로 인해서, two-stage detectors 가 object detection 방식을 지배하게 되었다.



* Two-stage Detector:  

최근의 object detection에서의 dominant paradigm 은 two-stage approach이다. 1단계에서 sparse set한 후보를 제안하고 2단계에서는 분류를 한다. R-CNN은 2단계 분류기에서 convolutional network를 사용하여 성능면에서 큰 발전을 이루었다. Region Proposal Networks(RPN) 과 2단계의 classfier 와 single convolution network 으로 통합되어 Faster R-CNN framework 가 만들어졌다.



* One-stage Detectors: 

OverFeat은 딥러닝을 사용한 최초의 one-stage object detector다. 최근에는 SSD, YOLO 등이 one-stage method 로서 좋은 성능을 냈다.  이들 detectors 는 speed 에 초점을 맞추었지만 성능도 나쁘지 않았다. SSD는 단지 AP가 10-20% 가 낮다. YOLO는 스피드/정확도 trade-off 에 더욱 집중했다. RetinaNet은 기존의 dense detectors 와 매우 유사하다. RPN에서 소개된 'anchor' 와 SSD와 FPN에서 사용한 features pyramids 를 사용한다. 중요한 것은 RetinaNet의 높은 성능을 보이는 이유는 네트워크 디자인이 아니라 새로운 loss 함수 때문이라는 점이다.



**Class Imbalance**: boosted detector,DPMs,최신의 SSD 같은 일반적인 one-stage object detection 에서는 학습 과정에서 클래스 불균형 문제가 심각했다.

이들 detectors 는 한장의 이미지에서 10^4~10^5 개의 후보 위치를 제안하지만 실제로는 몇개의 오브젝트만 있을 뿐이다.
이러한 불균형 현상은 2가지 문제를 일으킨다

(1) 분류하기 쉬운 easy negative 들이 대부분인데 이들이 학습에 기여하는 것이 거의 없기 때문에 학습이 비효율적이다.

(2) 수많은 easy negative 들이 학습 과정을 압도하므로 비일반적인 모델이 학습되어 버린다.

일반적인 해결방법은 학습과정에서 hard examples을 샘플링하는 hard negative minining(학습이 잘 안되는 샘플들을 모으는것) 을 하거나 더 복잡한 샘플링 기법 혹은 가중치 조절 방법을 쓰는 것이다.

반면, 우리가 제안하는 focal loss 는 one-stage detector 에서는 피할 수 없는 클래스 블균형문제를 자연스럽게 해결한다.

샘플링 기법을 사용하지 않고도 모든 샘플에 대해서 효율적인 학습이 이루어지도록 해준다.

focal loss는 easy negative 에게 압도당하지 않는 로스와 그라디언트를 계산해준다.


**Robust Estimation**: 매우 큰 에러를 갖는(hard example) outliers 의 loss 를 down-weighting 시킴으로써 loss 에 대한 영향력을 줄여주는 robust loss function(Huber loss) 은 sampling 기법보다 더욱 흥미로워 보인다. 

outlier 에 대한 접근하는 robust loss function 과는 반대로 focal loss 는 inliers(easy examples) 을 down-weiginting 시키므로서 그들의 영향력을 줄여준다.

비록 하나하나의 inliers 들의 loss 는 작지만 inliers들이 매우 많기 때문에 total loss에 끼치는 영향력은 크다.

다른 말로 하자면 focal loss는 robust loss 와 정반대로 동작한다고 할 수 있다.

focal loss 는 일부의 hard examples 에 대해 집중해서 학습한다.


### 3. Focal Loss

Focal Loss 는 one-stage object detection 에서 foreground 와 background 의 클래스간 불균형이 극도로 심한 상황(예를 들면 1:1000)을 해결하기 위해 디자인되었다.

binary classification을 위한 cross entropy(CE) loss 에서부터 focal loss 의 소개를 시작한다.

<img src='./imgs/retina3.png'>

(1) 에서 $y$ 는 1 or -1 이며 ground-truth class 를 가리킨다. $p$ 는 0 or 1 의 값을 갖으며 $y=1$인 클래스에 대해서 모델이 예측한 클래스 확률이다. 조금 더 편리하게 식을 다루기 위해서 (2)와 같이 $P_t$ 항을 정의하겠다.

<img src='./imgs/retina4.png'>

이므로 $CE(p,y) = CE(P_t) = -log(P_t)$ 이다.

CE loss 는 그림1의 파란색 커브의 형태를 갖는다.

CE loss의 가장 눈에 띄는 특징은 쉽게 예측된 샘플들(Pt >> 0.5)의 로스가 그다지 작지 않다는 것이다.

그 결과 이들 loss가 커지게 되면(혹은 많아지게 되면) rare(드문) class 들의 loss 를 압도해 버리는 상황이 생길 수 있기 때문에 문제가 된다.



#### 3.1 Balanced Cross Entropy

클래스 불균형을 해결하는 가장 일반적인 방법은 0~1 사이의 가중치 a(알파) 를 사용하는 것이다.

클래스 1 에는 a 를, 클래스 -1 에는 1-a 를 적용한다.

실제로 a 는 클래스 빈도의 역수(1:90 클래스 비율의 경우 a=90/1=90)를 사용하거나 하이퍼파라미터로서 크로스 밸리데이션과정을 통해 직접 값을 결정한다.

클래스 t 에 대한 알파 t 를 다음과 같이 a-balance CE loss 식으로 정의한다.

$$ CE(p_t) = -\alpha_t log(p_t) (3)$$

이 로스함수는 단순히 CE 를 확장한 것으로 실험에서는 이 식을 기본적으로 사용했다.

#### 3.2 Focal Loss Definition

클래스 dense detector의 학습과정에서 불균형 문제가 심각한 경우에는 (easy samples 에 의해) 크로스 엔트로피 로스의 압도 현상이 발생한다.
예측하기 쉬운 negative 샘플들이 대다수를 이루게 되면서 gradient 에 막대한 영향을 준다.

위 3.1에서 balance 파라미터인 $\alpha$는 positive/negative example의 중요한 정도를 밸런스가 맞도록 조절하는 역활을 한다.

그러나 알파는 easy/hard example 을 따로 구분하지 않는다.

반면 우리가 제안하는 변형된 loss function 은 easy example 의 중요도를 down-weight 시키며 그 결과 hard negative 의 학습에 집중하게 만든다.

cross entropy loss 에 조절 항 $(1 - p_t)^\gamma$ 을 추가했다. 

$\gamma$는 조절가능한 focusing parameter 로 0 이상의 값으로 셋팅한다.

다음과 같이 focal loss 를 정의한다.

$$ FL(p_t) = -(1 - p_t)^\gamma log(p_t) (4) $$

이 focall loss 에서 $\gamma$를 0~5 사이의 값으로 조절했을 때의 함수 형태를 그림 1 에서 볼 수 있다.


(1) 만약 example 을 잘못 분류했는데, $p_t$ 가 작은 값인 경우 조절항이 1에 가까워지면서 조잘 항의 값이 거의 1이 되므로 결국 로스 함수에 영향을 주지 않는다.

$$(1-0.00001)^\gamma \approx 1 $$

$Pt \approx 1$ 에 가까워 지면, 조절 항은 거의 0 이 된다. 
$$ (1-0.999)^\gamma \approx 0 $$

이것이 잘 분류된 샘플을 down-weight 시키는 경우이다.


(2) focusing parameter $\gamma$ 는 부드럽게 easy example을 down-weight 시킨다.

만약 $\gamma = 0$ 이라면, FL 은 CE 와 같다.

$\gamma$ 의 값이 커질수록 FL 의 down-weight 시키는 영향력이 커진다.(실험에서는 $\gamma = 2$인경우 가장 성능이 좋았다)

직관적으로 보면, modulating factor 은 easy example들의 loss 에 대한 영향력을 줄이며, loss 를 낮추는 example 의 범위를 결정한다.

예를 들어 $\gamma = 2$ 인 경우 $p_t = 0.9$ 로 예측되었던 example 의 원래 CE 로스에 비해서 FL 에서는 CE의 $p_t \approx 0.968$ 일때 수준의 작은 loss를 받게 되므로 상대적으로 1000x 작아 진다.

이것은 잘못  분류되었던 examples 의 중요도를 높이는 역활을 한다.

($\gamma = 2$ 로 셋팅되었을 때, $p_t \leq 0.5$ 로 예측된 example 의 loss 는 단지 4x 작아질 뿐이다.)


실제로 우리는 알파-balance variant 의 focal loss 를 다음과 같이 정의했다.

$$ FL(p_t) = -\alpha_t (1 - p_t)^\gamma log(p_t) (5) $$


위 로스 함수를 모든 실험에 적용했는데, 알파-balacned 항을 쓰지 않을 때에 비해 약간의 성능 향상이 있었다.

마지막으로, loss layer 에서 $p$를 계산하는 과정에서 sigmoid operation 을 사용할 때 굉장히 큰 성능 향상을 보였다.

그러나 전체적으로는 loss layer 를 무엇을 사용하던지간에 결과에 큰 차이는 없었다.

#### 3.3 Class Imbalance and Model Initialization

기본적으로 Binary classificatino Model 은 클래스 $y$가 -1, 1일 확률이 같도록 초기화된다.

이렇게 초기화했을 때 클래스 불균형 현상이 나타나면 더 많은 class 가 total loss 에서 더 많은 비중을 차지하게 되어 학습이 초기부터 불안정해진다.

이 문제를 해결하기 위해 'prior' 라는 개념의 $p$ 항을 사용하였다.

$p$ 항은 rare class(foreground) 의 비율로 최초 학습시에 사용하는 값이다.

rare class 의 비율이 0.01 처럼 낮은 경우 이 비율을 $p$라고 추정하며 파이에 의해 정의된다.

$p$ 항은 모델 초기화에서 사용되는 것일 뿐이고 loss function 이 아니다.

클래스 불균형문제가 심각한 경우에 $p$항을 사용하면 cross entropy나 focal loss 에 관계 없이 학습 안정화에 도움을 준다.


#### 3.4 Class Imbalance and Two-stage Detectors

Two-stage Detectors 는 보통 cross entropy loss 를 사용하고 알파-밸런싱은 사용하지 않는다.

대신 클래스 불균형 문제를 해결하기 위해 2가지 메카니즘을 사용한다.

(1) two-stage cascade  (2) baised minibatch sampling 

1번째 cascade stage 는 object proposal mechanism 으로 거의 무한대의 가능한 오브젝트의 위치를 1~2천개 정도로 줄이는 것이다.

선택할 proposals 은 물론 랜덤한 것이 아니라 true object location 에 가까운 것이어야만 한다.

그렇게 함으로써 대부분을 차지하는 easy negatives 를 제외할 수 있다.

2 단계 학습에서는, biased sampling 으로 minibatch 를 구성한다.

positive/negative 비율을 알파-밸런싱 factor 와 비슷하게 샘플링한다.

우리가 제안하는 focal loss 는 one-stage detection syste에서 loss function을 통해서 이러한 메카니즘들을 직접 디자인한다.


### 4. RetinaNet Detector

RetinaNet은 하나로 통합 네트워크로 backbone network 와 2개의 전용 subnetwork 로 구성된다.

backbone 은 입력 이미지 전체 영역에 대해서 convolutional feature map 을 계산하는 역활이며, 누구나 사용할 수 있는 공개된 convolutional network 이다.

첫번째 subnet 은 backbone 의 output으로 부터 convolutional 을 통해서 object classification을 수행한다.

두번째 subnet 은 backbone 의 output으로 부터 convolutional 을 통해서 경계박스의 좌표(앵커와 정답간의 offset)를 구한다.

2개의 subnetwork의 디자인은 간단하다. 

이것으로 그림 3과 같이 one-state, dense detection 을 구성했다.

설계에는 많은 선택사항이 있지만, 실험 결과처럼 대부분의 디자인 파라미터들은 성능에 대해 크게 민감하지 않다.

<img src='./imgs/retina5.png'>

backbone 은 ResNet architecture를 사용해서 (a)의 rich,multi-scale convolutional feature pyramid를 생성한다.

(b) backbone 에 2개의 subnetwork 를 붙여서 RetinaNet이 된다. 하나는 앵커 박스의 클래스 분류, 다른 하나는 앵커 박스와 ground-truth object boxes간의 거리를 예측한다.

(d) 네트워크 디자인은 일부러 간단하게 했다. 

우리의 목표는 새로운 네트워크 구조를 제안하는 것이 아니다.

focal loss function 를 통해서 one-stage detector 와 그동안 최고 성능을 보인 Faster R-CNN with FPN 등의 two-stage detector간의 성능 차이를 줄이는 것, 그리고 빠르게 동작시키는 것이다.



#### Feature Pyramid Network Backbone: 

Feature Pyramid Network(FPN)을 RetinaNet의 backbone 으로 채택했다. 간단하게 말하면, FPN은 일반 convolutional network를 top-down 방향으로 augment 한다. 

그림 (a)-(b) 와 같이 lateral connection(수평 연결) 을 통해서 하나의 resolution 입력 이미지로부터 rich한 multi-scale feature pyramid 를 효율적으로 구성한다.

각각의 피라미드 레벨은 서로 다른 크기의 object 를 검출하는데 사용한다.

FPN은 fully convolutional network(FCN) 에 비해서 multi-scale 예측 성능이 더 좋다는 것이 이미 RPN, DeepMask-style proposal, two-stage detector 인 Fast R-CNN 이나 Mast R-CNN 등을 통해서 알려져 있다.

ResNet 구조 위에 FPN 을 구성하였다.

피라미드 레벨을 P3에서 P7이 되도록 했다. 

숫자의 의미는 입력 이미지 크기에 대해 2의 승수만큼 작다는 것이다.(P1 = input size, P2 = input size/2, P3 = input size/2/2 ...)

모든 피라미드 레벨의 채널 C는 256 으로 동일하다. 

설계에서 다양한 디자인을 선택할 수 있었지만 성능에 큰 영향을 주지는 않는다. 

다만 FPN을 backbone 으로 쓰는 것은 중요하다.

FPN 없이 ResNet 의 최종 레이어만을 사용해 예측하는 경우에는 AP 가 낮았다.


#### Anchor: 

RPN(Region Proposal Network) 에서 사용하는 translation-invariant anchor box(이동 불변 앵커박스) 를 사용했다.

앵커박스의 크기는 피라미드 레벨 P3~P7에서 32^2 ... 512^2 의 크기를 갖는다.

각 피라미드 레벨에서 3개의 종횡비{1:2, 1:1, 2:1} 를 갖는 앵커박스를 사용했다.

모든 스케일에 대응(denser scale coverage) 하기 위해서 각 피라미드 레벨에서 원래의 3개의 앵커박스에 대해서 각각 3개를 더 추가했다 {2^0, 2^1/3, 2^2/3}

이렇게 하는것이 AP 를 올리는데 도움이 되었다.

각 레벨에서 총 A = 9 개의 앵커박스를 갖고 있으며 입력이미지의 32 - 813 픽셀까지 담당할 수 있다.

각 앵커박스에는 K개의 one-hot vector classification target 와 4-vector의 box regression target 이 할당된다.

K 는 number of ojbect class 이다.

GT와의 intersection-over-union(IoU)가 0.5 이상인 앵커박스만 ground-trugh object box 로 할당된다.

IoU 가 0.4 이하의 앵커박스는 background 로 할당된다.

앵커박스는 최대(오직) 1개의 object box 에만 할당된다.

K label vector 는 클래스에 해당하는 entry만 1, 나머지는 0으로 채운다.(one-hot)

만약 앵커박사가 아무것에도 할당되지 않는다면, 즉 GT와의 IoU가 0.4~0.5 인 경우라면, 해당 앵커박스는 학습과정에서 무시된다.

Box regression targets 은 각 앵커박스와 할당된 object box 와의 offset(거리)를 계산한다.

할당되지 않은 앵커박스들은 계산에서 빠진다.



#### Classification Subnet: 

classification subnet은 각 위치에서 A 앵커에의 K개의 object class 확률을 예측한다

이 subnet은 각 FPN 레벨에 붙은 작은 FCN 이다.

subnet의 네트워크 구조는 모든 피라미드 레벨에서 전부 동일하다. 

디자인도 매우 단순하다. 

그림 3 (c) 와 같이 주어진 피라미드 레벨에서 C 채널의 feature map을 받는다.

3x3 컨볼루션을 256 개의 필터로 컨볼루션하고 ReLU 활성함수를 적용한다. 이것을 4회반복 (x 4)

마지막으로 KA(K*A) 개의 필터로 컨불루션한 후 매 공간 위치에서 binary prediction을 위해서 Sigmoid 활성함수를 적용한다.

(Sigmoid 와 FL  조합의 성능이 좋았다)



RPN 과는 대조적으로 우리의 object classification subnet은 레이어가 깊고 3x3 컨볼루션만 사용한다.

모든 피라미드 레벨에서 파라미터를 공유한다.

<img src='./imgs/retina6.png'>
              

참고) RPN과 RetinaNet RPN의 클래스,경계박스 예측 레이어 : RPN에서는 cls layer와 reg layer가 intermediate layer의 출력 feature map을 공유한다



이렇게 전체 레벨에서 디자인 하는 것이 하이퍼 파라미터를 조절하는 것보다 더 중요하다는 것을 확인했다.



#### Box Regression Subnet: 

각 피라미드 레벨에서 object classification subnet과는 병렬적으로 또다른 작은 FCN을 붙였다.

이것은 그림 3(d) 와 같이 각 앵커박스에서 근처에서 ground-truth object 가 있는 경우 둘 간의 offset 을 예측하는 것이 목표이다.

각각의 위치에서의 A 개의 앵커박스들은 자신과 GT간의 상대적인 좌표 offset 4개를 예측한다.

(R-CNN 에서 사용하는 일반적인 박스 파라미터를 사용했다. cx, cy, w, h )

최근의 work 와는 다르게, 우리는 클래스에 관계없이 좌표를 예측하도록 했는데, 성능의 차이가 없음을 확인했다.

(일부 최근의 work에서는 클래스마다 좌표 예측을 하는 파라미터들이 다르다).

object classification subnet 과 box regression subnet은 모두 공통의 구조를 가지지만, box regression subnet은 파라미터는 공유하지 않는다.




#### Inference: 
속도를 위해서 detector threshold 0.05로 걸러낸 이후 top-scoring proection 1k 개의 box prediction 만 사용했다.

모든 피라미드 레벨에서의 예측들은 최종 detection 을 위해서 threshold 0.5의 non-maximum suppression 을 통해서 중복 예측들이 통합된다.



#### Focal Loss: 

classification subnet의 output 에 focal loss 를 사용했다.

$\gamma$ 가 0.5~5 일때 모두 괜찮지만 $\gamma$ = 2 일때 가장 성능이 좋다.

RetinaNet을 학습할때, focal loss에서 계산되는 anchor의 갯수가 all~100k 개의 합이고 각 로스들이 앵커와 GT박스의 갯수의 비율로 정규화된다는 점을 강조하고 싶다.

전체 앵커박스가 아니라 할당이 된 앵커박스의 수로 정규화했다.

그 이유는 대부분의 앵커박스는 easy negative 여서 focal loss를 사용하게 되면 매우 작은 loss 를 받기 때문이다.

마지막으로, rare class 에 할당되는 weight 인 알파도 역시 그리 민감하지 않다.

Table 1a와 1b 에서 감마와 알파 값에 따른 성능이 나타나있다.

일반적으로 감마값을 키울수록 알파값은 감소시켜야 한다.

(감마=2, 알파=0.25가 가장 좋았다)



#### Initialization:

ResNet-50-FPN 과 ResNet-101-FPN 두개를 backbone으로 실험했다.

둘 다 ImageNet1k 로 pre-train 시켰다. 

'Feature pyramid networks for object detection'처럼 FPN을 위해 새로운 레이어를 추가했다.

최종 레이어를 제외하고는 bias b = 0, Gaussian weight fill with 델타 = 0.01 로 초기화했다.

classification subnet의 마지막 레이어의 bias 는 b = -log((1-파이)/파이) 로 초기화 했다.

파이는 최초 학습에서 모든 앵커의 라벨의 foreground 에 대한 confidence 값으로 모든 실험에서 0.01을 사용했다.

이렇게 초기화하면 너무 많은 background anchor 가 생성되는 것을 방지해 주기 때문에 첫번째 iteration에서 학습이 안정된다.



#### Optimization: 

RetinaNet 은 stochastic gradient descent(SGD)로 학습된다. 

8개의 GPU에서 synchronized SGD 를 사용했고 GPU 당 배치 2개이므로 전체 총 배치는 16개다.

(개별 GPU에서 개별 배치에 대해 Gradient를 계산 - 동기화 - 모든 Gradient를 모아 평균내서 parameter를 update - 개별 GPU에 update된 parameter 전달 - 동기화)

모든 모델은 90k iteration 이고, 학습율은 0.01 이고 60k 와 80k 에서 10으로 나눈다.

augmentation은 horizontal image flipping 만을 사용했다.

Weight decay 는 0.0001, momentum 0.9 다.

training loss 는 focal loss와 box regression의 standard smooth L1 loss 의 합이다

학습 시간은 모델에 따라서 10~35시간으로 Table 1e 에 나타나 있다.
